<a href="https://colab.research.google.com/github/ilia-kuznetsov-hds/machine_learning_project/blob/main/2025_11_MLII_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2 - Diagnosis coding from clinical summary using prompt engineering



---


#####################################################################################

**[Double-click to write down your zID, Name and Surname]**

Write your zID below and replace the placeholder “zID”, "Name" and "FamilyName" in the Jupyter Notebook name with your actual zID, name and family name. That is, instead of `zID-Project-02-Name-FamilyName.ipynb`, write your actual zID, Name and Family Name.

**zID:** z5567497

**Name:** Ilia

**Surname:** Kuznetsov

**Honour Pledge** <p>
    
Declaration: <p>
    
    
I declare that this assessment item is **my own work**, except where acknowledged, and has not been submitted for academic credit elsewhere or previously, or produced independently of this course (e.g. for a third party such as your place of employment) and acknowledge that the assessor of this item may, for the purpose of assessing this item:

1. Reproduce this assessment item and provide a copy to another member of the University; and/or

2. Communicate a copy of this assessment item to a plagiarism checking service (which may then retain a copy of the assessment item on its database for the purpose of future plagiarism checking).

**By writing your zID, name and surname above, you certify that you have read and agreed to the honour pledge.**

#####################################################################################



---



---



---



Please write the questions as they are written in the [Word document](https://unsw-my.sharepoint.com/:w:/g/personal/z3368601_ad_unsw_edu_au/ERY2EhDgUPxAjrX0UFC_dHsBcc1e3hVtIxeWSIu-PJp5OA?e=iiAnOh) and your answers here. Create as many text or codes cells as needed.

## Model Loading

Run the cell below to download the `Qwen/Qwen2.5-0.5B-Instruct` model from huggingface, and running it on CPU.

DO NOT modify this cell.



In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    dtype="auto",
    device_map="cpu" # Using CPU
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

## Dataset creation instruction

**Development set `patient_id`: 825, 1411, 4399, 4644, 5353**

**Test set ``patient_id``: 418, 608, 2678, 3824, 3972, 4046, 4175, 4679, 4758, 5545**

**Ground truth for the development set:**



*   Patient_id: 825. ICD1-10-AM: G05
*   Patient_id: 1411. ICD1-10-AM: A52
*   Patient_id: 4399. ICD1-10-AM: K75
*   Patient_id: 4644. ICD1-10-AM: S83
*   Patient_id: 5353. ICD1-10-AM: L03



## Evaluation Data Frame

The code in the following cell creates the pandas DataFrame containing the ground-truth labels (ICD-10-AM principal diagnosis codes) for evaluation in Question 4.

DO NOT modify this cell; Run it to create the `Eval` DataFrame for Question 4.

In [2]:
import pandas as pd

data = [
    (418,  "C79"),
    (608,  "M85"),
    (2678, "C49"),
    (3824, "I97"),
    (3972, "A18"),
    (4046, "N49"),
    (4175, "K25"),
    (4679, "D16"),
    (4758, "J85"),
    (5545, "K42"),
]


Eval = pd.DataFrame(data, columns=[
    "patient_id",
    "ICD-10-AM principal code"
])


Eval["Model generated ICD-10-AM Code"] = ""

# Question 1 – Understanding ICD-10-AM Coding and Developing Your Research

- What is ICD-10-AM diagnostic coding, and what is ICD-10-AM? Write up to 300 words.

- What is the principal diagnosis code (in the context of ICD-10-AM)? Write up to 100 words. You may refer to the following resource for guidance: https://ar-drg.laneprint.com.au/wp-content/uploads/2020/10/ACS-Sample.pdf

- What is your research question for this project? Write up to 50 words

Research question: “How does the choice between structured template prompts, few-shot exemplars, and chain-of-thought prompts influence the accuracy of Qwen/Qwen2.5-0.5B-Instruct in predicting ICD-10-AM principal diagnosis codes from hospital-course summaries in the Asclepius Synthetic Clinical Notes dataset?”

# Question 2 – Understanding the Asclepius dataset:

Before prompting any LLMs, it is essential to understand the data you are working with. Conduct background research on the Asclepius dataset;   
Your answer should include, but is not limited to, the following points:

- What is the original source?

- How did the author create this dataset from its source?

- What is the overall structure of the dataset?

- What is the purpose of this dataset?

# Question 3 – LLM prompt engineering

Your task is to create and test a range of prompts for running inference on the designated LLM (Qwen/Qwen2.5-0.5B-Instruct) in a Google Colab environment.

§ Use the hospital course summary as the input to your prompt.

§ Your goal is to generate the corresponding principal diagnosis ICD-10-AM code.

§ Your prompt must produce exactly one ICD-10-AM code in the model’s response for each admission.

§ You must output only the ICD-10-AM category, i.e., the first three characters of the code (one letter followed by two numbers).

§ You should experiment with several prompt-engineering techniques introduced in class and explore different inference hyperparameters.

§ For this question, you must use only the development set when designing and refining your prompts (similar to working with a combined training–validation set).

You will not be assessed on ICD-10-AM coding accuracy, but rather on the quality of your prompt-development process and your understanding of LLM inference.

Keep a record of all prompts you tried as evidence of your prompt-development process. Select the five that best represent your approach and rationale, and include these in Question 3 of your Jupyter Notebook. Any additional prompts should be placed in an appendix at the end of the notebook, accompanied by comments and text cells explaining them.

In [3]:
# Install dependencies into the venv

%pip install datasets huggingface_hub

#
from datasets import load_dataset
from pprint import pprint
ds = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes")
print(ds)
for split in ds:
    print(split, len(ds[split]))
print("Features for first split:")
pprint(ds[list(ds.keys())[0]].features)
print("Example 0:")
pprint(ds[list(ds.keys())[0]][0])


README.md: 0.00B [00:00, ?B/s]

synthetic.csv:   0%|          | 0.00/402M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/158114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'note', 'question', 'answer', 'task'],
        num_rows: 158114
    })
})
train 158114
Features for first split:
{'answer': Value('string'),
 'note': Value('string'),
 'patient_id': Value('int64'),
 'question': Value('string'),
 'task': Value('string')}
Example 0:
{'answer': 'The healthcare team used a gradual approach to changing the '
           "patient's position to avoid worsening of the respiratory status "
           'and prevent respiratory failure.',
 'note': 'Discharge Summary:\n'
         '\n'
         'Patient: 60-year-old male with moderate ARDS from COVID-19\n'
         '\n'
         'Hospital Course:\n'
         '\n'
         'The patient was admitted to the hospital with symptoms of fever, dry '
         'cough, and dyspnea. During physical therapy on the acute ward, the '
         'patient experienced coughing attacks that induced oxygen '
         'desaturation and dyspnea with any change of position 



---



---



---



© 2025 Copyright The University of New South Wales - CRICOS 00098G